# Hyper Parameter Search

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)


import numpy as np
import pandas as pd
from src.trainer import Trainer
from src.trainer import CaseDataSet
from src.model import DLModels

import torch
from torch import nn

import optuna
import importlib.util
import joblib
import pickle

/home/ap43owus/anaconda3/envs/tl4pm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trainer_hyper_para = {"max_epoch": 100,
                      "max_ob_iter": 20,
                      "score_margin": 1,
                      "num_class": 1,
                      "num_layers": 1,
                      "learning_rate": 1e-3,
                      "batch_size": 1000,
                      "training_loss_func": "CCM",
                      "eval_loss_func": "CCM"}

model_hyper_para = {"w2v": {"input_size": 51,
                            "hidden_size": 128,
                            "num_layers": 1},
                    "st": {"input_size": 385,
                           "hidden_size": 512,
                           "num_layers": 1},
                    "onehot": {"input_size": 48,
                               "hidden_size": 128,
                               "num_layers": 1}}


split_pattern = ["811split", "641620split"]
data_set_type = ["source", "target"]
data_set_cat = ["train", "val", "test"]
embedding_type = ["w2v", "st", "onehot"]
dataset_list_index = [split_pattern, embedding_type, data_set_type, data_set_cat]

earliness_requirement = True
folder_path = "../../Data/Training/"

In [3]:
def get_torch_device():
    torch_device = "cpu"
    device_package = torch.cpu
    if importlib.util.find_spec("torch.backends.mps") is not None:
        if torch.backends.mps.is_available():
            torch_device = torch.device("mps")
            device_package = torch.mps
    if torch.cuda.is_available():
        torch_device = torch.device("cuda")
        device_package = torch.cuda
    return torch_device, device_package

In [4]:
torch_device, device_package = get_torch_device()

train_set = CaseDataSet.CaseDataset(split_pattern="641620split", input_data="source",
                                    data_version="_train_sorted", embedding_version="_st",
                                    earliness_requirement=earliness_requirement)


val_set = CaseDataSet.CaseDataset(split_pattern="641620split", input_data="source",
                                  data_version="_val_sorted", embedding_version="_st",
                                  earliness_requirement=earliness_requirement)

In [25]:
def objective(trial):
    # Hyperparameters
    input_size = 385  # The number of expected features in the input x
    hidden_size = trial.suggest_int("n_hidden", 4, 512)  # The number of features in the hidden state h
    num_layers = trial.suggest_int("n_layer", 1, 4)  # Number of recurrent layers
    batch_size = trial.suggest_int("batch_size", 10, 10000)
    num_classes = 1  # For binary classification
    learning_rate = 0.001

    model = DLModels.SimpleLSTM(input_size, hidden_size, num_layers, num_classes).to(torch_device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model, train_loss_source, val_loss_srouce = Trainer.train_model(model, optimizer,
                                                                   None, None,
                                                                   train_set,
                                                                   val_set,
                                                                   batch_size,
                                                                   torch_device,
                                                                   device_package,
                                                                   Trainer.prefix_weighted_loss,
                                                                   trainer_hyper_para["max_epoch"],
                                                                   trainer_hyper_para["max_ob_iter"],
                                                                   trainer_hyper_para["score_margin"],
                                                                   print_iter=False)
    
    
    model.flatten()
    roc_auc, f1, f1_inverse, precision, precision_inverse, recall, recall_inverse = Trainer.eval_model(model, val_set,
                                                                                                           torch_device=torch_device,
                                                                                                           device_package=device_package)
    model_name = "LSTM_S_h" + str(hidden_size) + "_l" + str(num_layers) + "_st"
    torch.save(model, "../../Model/Optuna/641620split/LSTM/"+ model_name + ".LSTM")
    
    training_stat = pd.DataFrame(columns=["TrainingLoss", "ValidationLoss"],
                                 data=np.hstack([train_loss_source.reshape((-1, 1)),
                                                 val_loss_srouce.reshape((-1, 1))]))
    training_stat.to_pickle("../../Model/Optuna/641620split/LSTM/"+ model_name + "_S_st_stat.pkl")
    
    del model
    device_package.empty_cache()
    return roc_auc

In [26]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)
joblib.dump(study, "../../Model/Optuna/641620split/LSTM/Study_source_641620split_st.pkl")

[I 2024-03-20 12:45:53,923] A new study created in memory with name: no-name-5230a196-1e6d-4eb4-ae2d-1fc77bc4fbf8
[I 2024-03-20 12:56:24,427] Trial 0 finished with value: 0.71385180313697 and parameters: {'n_hidden': 466, 'n_layer': 4, 'batch_size': 3989}. Best is trial 0 with value: 0.71385180313697.
[I 2024-03-20 13:13:04,978] Trial 1 finished with value: 0.7452219090403807 and parameters: {'n_hidden': 175, 'n_layer': 3, 'batch_size': 9844}. Best is trial 1 with value: 0.7452219090403807.
[I 2024-03-20 13:25:48,909] Trial 2 finished with value: 0.7581979965650995 and parameters: {'n_hidden': 423, 'n_layer': 2, 'batch_size': 4358}. Best is trial 2 with value: 0.7581979965650995.
[I 2024-03-20 13:28:32,476] Trial 3 finished with value: 0.7516930264455015 and parameters: {'n_hidden': 33, 'n_layer': 2, 'batch_size': 2437}. Best is trial 2 with value: 0.7581979965650995.
[I 2024-03-20 13:30:55,307] Trial 4 finished with value: 0.7492392833563173 and parameters: {'n_hidden': 18, 'n_layer':

KeyboardInterrupt: 

In [28]:
study = joblib.load("../../Model/Optuna/641620split/LSTM/Study_source.pkl")

In [34]:
study.StudySummary()

AttributeError: 'Study' object has no attribute 'StudySummary'